In [21]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import LSTM, Activation, Dropout, Embedding, Input
from keras import regularizers
from keras.optimizers import Nadam, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization

import math, json

import pandas as pd
import numpy as np

import os.path

In [22]:
def import_log(filepath):
    df = pd.read_csv(filepath)
    return(df.values.tolist())

def remove_nan(lists):
    newlists = []
    for tr in lists:
        newlists.append([int(x) for x in tr if str(x) != 'nan'])
    return(newlists)

def create_XY_prefix(log, mappingsize, prefixlen):
    X = []
    Y = []
    for i in range(0, len(log)):
        for k in range(1, len(log[i])):
            X.append(log[i][max(0, k-prefixlen):k]) #get the prefix of 'encoded' activities
            y = [0] *(mappingsize)
            y[int(log[i][k])-1] = 1
            Y.append(y)        
    X = keras.preprocessing.sequence.pad_sequences(X, maxlen=prefixlen, padding='pre')
    return(np.array(X), np.array(Y))

def get_model_onedirectional(maxlen, num_chars, embedding, n_layers, lstmsize, dropout, l1, l2):
    emb_size = math.ceil(num_chars**0.25)
    model = Sequential()
    if embedding == True: #if you use embedding layer input should be number encoded
        model.add(Input(shape=(maxlen, )))
        model.add(Embedding(input_dim=num_chars+1, output_dim=emb_size))
    else:
        model.add(Input(shape=(maxlen, num_chars))) #If you don't use an embedding layer input should be one-hot-encoded
    model.add(LSTM(lstmsize,
                   kernel_initializer='glorot_uniform',
                   return_sequences=(n_layers != 1),
                   kernel_regularizer=regularizers.l1_l2(l1,l2),
                   recurrent_regularizer=regularizers.l1_l2(l1,l2),
                   input_shape=(maxlen, num_chars)))
    model.add(Dropout(dropout))
    for i in range(1, n_layers):
        return_sequences = (i+1 != n_layers)
        model.add(LSTM(lstmsize,
                       kernel_initializer='glorot_uniform',
                       return_sequences=return_sequences,
                       kernel_regularizer=regularizers.l1_l2(l1,l2),
                       recurrent_regularizer=regularizers.l1_l2(l1,l2)))
        model.add(Dropout(dropout))
    
    model.add(Dense(num_chars, kernel_initializer='glorot_uniform',activation='softmax'))
    opt = Adam(learning_rate=0.005)
    model.compile(loss='categorical_crossentropy', optimizer=opt)
    return model

def get_startend(log): #get first and last activity, we use it to stop the simulation. Should be changed!!!
    return log[0][0], log[0][-1]

def get_probabilities(rnnmodel, xlists,  nr_act, maxlen, prefixlen):
    #assume xlist is a list with the x (prefix) untill now 
    all_x = keras.preprocessing.sequence.pad_sequences(xlists, maxlen=maxlen, padding="pre")
    all_x = all_x[:,-(prefixlen):]
    results = rnnmodel.predict(all_x)
    return results

def normalize(probs): #normalize probabilities to sum to 1
    examplesize, actsize = probs.shape
    newy = []
    for i in range(examplesize):
        normalizer = 1 / float( sum(probs[i]) )
        ynorm = [float(l) * normalizer for l in probs[i]]
        newy.append(ynorm)
    return newy


def choose_act_all(all_y): #randomly choose an activity, stochastically
    #p want a list of probabilities    
    chosen_acts = []
    for i in range(len(all_y)):
        chosen_acts.append(np.random.choice(np.arange(0, len(all_y[i])), p=all_y[i])+1)  
    return(chosen_acts)   # +1 because number encodig starts at 1 not 0

def cut_end(log, endact):
    logsize, tracesize = log.shape
    print(log.shape)
    newlog = []
    for i in range(0, logsize):
        trace = []
        for j in range(0, tracesize):
            if log[i][j] == endact:
                trace.append(log[i][j])
                break
            else:
                trace.append(log[i][j])
        newlog.append(trace)
    return(newlog)


def simulate_log(RNNmodel, logsize, startact, endact, maxlen, mapping, prefixlen): #Use RNN to simulate log
    log = np.zeros((logsize, maxlen+1), int)
    for i in range(0, logsize): #start every trace with the start activity
        log[i][0] = startact
    print(log)
    for j in range(1,maxlen+1): #check if 0 or 1 and ml or ml - 1 #we took 50 for with loops   
        print("finding activity nr", j+1)   
        prefixes = np.array([log[i][0:j] for i in range(0, logsize)])
        print(prefixes)
        probs = get_probabilities(RNNmodel, prefixes, len(mapping), maxlen, prefixlen)
        #we need to do this because otherwise probabilities sum over 1 
        ynorm = normalize(probs) 
        nextacts = choose_act_all(ynorm) 
        for i in range(0, logsize):
            log[i][j] = nextacts[i]
    print(log)
    corrected_log = cut_end(log, endact)      
    return(corrected_log) 

def save_log(loglist, filename): #save a list of lists 
    df = pd.DataFrame.from_records(loglist)
    df.to_csv(filename, index=False)

def number_to_one_hot_X(X, dict_size): #if we want 
    newX = []
    for example in X:
        new_ex = []
        for i in range(len(example)):
            onehot = [0]*dict_size #changed
            if example[i] != 0:
                onehot[example[i] - 1] = 1 #-1 because 
            new_ex.append(onehot)
        newX.append(new_ex)
    return(np.array(newX))


def OHget_probabilities(rnnmodel, xlists,  nr_act, maxlen, prefixlen):
    #assume xlist is a list with the x (prefix) untill now 
    all_x = keras.preprocessing.sequence.pad_sequences(xlists, maxlen=maxlen, padding="pre")
    all_x = all_x[:,-(prefixlen):]
    all_x = number_to_one_hot_X(all_x, nr_act)
    results = rnnmodel.predict(all_x)
    return results




def OHsimulate_log(RNNmodel, logsize, startact, endact, maxlen, mapping, prefixlen): #Use RNN to simulate log
    log = np.zeros((logsize, maxlen+1), int)
    for i in range(0, logsize): #start every trace with the start activity
        log[i][0] = startact
    print(log)
    for j in range(1,maxlen+1): #check if 0 or 1 and ml or ml - 1 #we took 50 for with loops   
        print("finding activity nr", j+1)   
        prefixes = np.array([log[i][0:j] for i in range(0, logsize)])
        print(prefixes)
        probs = OHget_probabilities(RNNmodel, prefixes, len(mapping), maxlen, prefixlen)
        #we need to do this because otherwise probabilities sum over 1 
        ynorm = normalize(probs) 
        nextacts = choose_act_all(ynorm) 
        for i in range(0, logsize):
            log[i][j] = nextacts[i]
    print(log)
    corrected_log = cut_end(log, endact)      
    return(corrected_log) 

In [23]:
def fit_models_prefix10(modelname):
    mappingfilename = 'Mappings/mapping_'+modelname+'.txt' 
    with open(mappingfilename) as f:
        mapping = json.loads(f.read())
    
    trainname = "Train_sets/log_" + modelname +'.csv'
    train_log = remove_nan(import_log(trainname))

    maxLength = max(map(len, train_log))

    X_train, y_train = create_XY_prefix(train_log, len(mapping), 10)
    X_train = number_to_one_hot_X(X_train, len(mapping))

    model = get_model_onedirectional(maxlen=10, num_chars=len(mapping), embedding=False, 
                                     n_layers=1, lstmsize=32, dropout = 0.4, 
                                     l1=0.001, l2=0.001)
    model.summary()
    batch_size = 128
    early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
    lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', 
                                   min_delta=0.0001, cooldown=0, min_lr=0)

    # train the model, output generated text after each iteration
    history = model.fit(X_train, y_train, validation_split=0.2,callbacks=[early_stopping, lr_reducer],
                        batch_size=batch_size, epochs=600, verbose=2)


    #save RNN model in case we ever need it
    modelfilename = 'Models/'+ modelname
    model.save(modelfilename)

    start,end = get_startend(train_log)


    simlog = OHsimulate_log(model, 12000, start, end, maxLength-1, mapping, 10)

    #save simulated log
    SimLogName = 'Sim_sets/Log_'+ modelname + '.csv'
    save_log(simlog, SimLogName)

In [28]:
def fit_models_prefix10_withloops(modelname):
    mappingfilename = 'Mappings/mapping_'+modelname+'.txt' 
    with open(mappingfilename) as f:
        mapping = json.loads(f.read())
    
    trainname = "Train_sets/log_" + modelname +'.csv'
    train_log = remove_nan(import_log(trainname))

    maxLength = max(map(len, train_log))

    X_train, y_train = create_XY_prefix(train_log, len(mapping), 10)
    X_train = number_to_one_hot_X(X_train, len(mapping))

    model = get_model_onedirectional(maxlen=10, num_chars=len(mapping), embedding=False, 
                                     n_layers=1, lstmsize=32, dropout = 0.4, 
                                     l1=0.001, l2=0.001)
    model.summary()
    batch_size = 128
    early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
    lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', 
                                   min_delta=0.0001, cooldown=0, min_lr=0)

    # train the model, output generated text after each iteration
    history = model.fit(X_train, y_train, validation_split=0.2,callbacks=[early_stopping, lr_reducer],
                        batch_size=batch_size, epochs=600, verbose=2)


    #save RNN model in case we ever need it
    modelfilename = 'Models/'+ modelname
    model.save(modelfilename)

    start,end = get_startend(train_log)


    simlog = OHsimulate_log(model, 12000, start, end, maxLength+15, mapping, 10) #+15 becaue we want to be able to let it play for some more

    #save simulated log
    SimLogName = 'Sim_sets/Log_'+ modelname + '.csv'
    save_log(simlog, SimLogName)

In [25]:
def fit_models_full_prefix(modelname):
    mappingfilename = 'Mappings/mapping_'+modelname+'.txt' 
    with open(mappingfilename) as f:
        mapping = json.loads(f.read())
    
    trainname = "Train_sets/log_" + modelname +'.csv'
    train_log = remove_nan(import_log(trainname))

    maxLength = max(map(len, train_log))

    X_train, y_train = create_XY_prefix(train_log, len(mapping), maxLength-1)
    X_train = number_to_one_hot_X(X_train, len(mapping))

    model = get_model_onedirectional(maxlen=maxLength-1, num_chars=len(mapping), embedding=False, 
                                     n_layers=1, lstmsize=32, dropout = 0.4, 
                                     l1=0.001, l2=0.001)
    model.summary()
    batch_size = 128
    early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
    lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', 
                                   min_delta=0.0001, cooldown=0, min_lr=0)

    # train the model, output generated text after each iteration
    history = model.fit(X_train, y_train, validation_split=0.2,callbacks=[early_stopping, lr_reducer],
                        batch_size=batch_size, epochs=600, verbose=2)


    #save RNN model in case we ever need it
    modelfilename = 'Models/'+ modelname
    model.save(modelfilename)

    start,end = get_startend(train_log)


    simlog = OHsimulate_log(model, 12000, start, end, maxLength-1, mapping, maxLength-1)

    #save simulated log
    SimLogName = 'Sim_sets/Log_'+ modelname + '.csv'
    save_log(simlog, SimLogName)

In [26]:
def fit_models_prefix10_complex(modelname):
    mappingfilename = 'Mappings/mapping_'+modelname+'.txt' 
    with open(mappingfilename) as f:
        mapping = json.loads(f.read())
    
    trainname = "Train_sets/log_" + modelname +'.csv'
    train_log = remove_nan(import_log(trainname))

    maxLength = max(map(len, train_log))

    X_train, y_train = create_XY_prefix(train_log, len(mapping), 10)
    X_train = number_to_one_hot_X(X_train, len(mapping))

    model = get_model_onedirectional(maxlen=10, num_chars=len(mapping), embedding=False, 
                                     n_layers=1, lstmsize=32, dropout = 0.4, 
                                     l1=0.001, l2=0.001)
    model.summary()
    batch_size = 128
    early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
    lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', 
                                   min_delta=0.0001, cooldown=0, min_lr=0)

    # train the model, output generated text after each iteration
    history = model.fit(X_train, y_train, validation_split=0.2,callbacks=[early_stopping, lr_reducer],
                        batch_size=batch_size, epochs=600, verbose=2)


    #save RNN model in case we ever need it
    modelfilename = 'Models/'+ modelname
    model.save(modelfilename)

    start,end = get_startend(train_log)


    simlog = OHsimulate_log(model, 66000, start, end, maxLength+15, mapping, 10)

    #save simulated log
    SimLogName = 'Sim_sets/Log_'+ modelname + '.csv'
    save_log(simlog, SimLogName)

In [9]:
fit_models_prefix10('Model1')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5888      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 13)                429       
Total params: 6,317
Trainable params: 6,317
Non-trainable params: 0
_________________________________________________________________
Epoch 1/600
720/720 - 20s - loss: 0.9059 - val_loss: 0.5651
Epoch 2/600
720/720 - 3s - loss: 0.6035 - val_loss: 0.5287
Epoch 3/600
720/720 - 4s - loss: 0.5723 - val_loss: 0.5009
Epoch 4/600
720/720 - 3s - loss: 0.5551 - val_loss: 0.4969
Epoch 5/600
720/720 - 3s - loss: 0.5536 - val_loss: 0.4980
Epoch 6/600
720/720 - 4s - loss: 0.5353 - val_loss: 0.4773
Epoch 7/600
720/720 - 3s - lo

Epoch 125/600
720/720 - 3s - loss: 0.4321 - val_loss: 0.4162
Epoch 126/600
720/720 - 3s - loss: 0.4314 - val_loss: 0.4166
Epoch 127/600
720/720 - 3s - loss: 0.4319 - val_loss: 0.4153
Epoch 128/600
720/720 - 3s - loss: 0.4316 - val_loss: 0.4156
Epoch 129/600
720/720 - 3s - loss: 0.4319 - val_loss: 0.4169
Epoch 130/600
720/720 - 3s - loss: 0.4312 - val_loss: 0.4154
Epoch 131/600
720/720 - 3s - loss: 0.4313 - val_loss: 0.4169
Epoch 132/600
720/720 - 3s - loss: 0.4308 - val_loss: 0.4175
Epoch 133/600
720/720 - 3s - loss: 0.4289 - val_loss: 0.4146
Epoch 134/600
720/720 - 3s - loss: 0.4289 - val_loss: 0.4147
Epoch 135/600
720/720 - 3s - loss: 0.4292 - val_loss: 0.4146
Epoch 136/600
720/720 - 3s - loss: 0.4291 - val_loss: 0.4147
Epoch 137/600
720/720 - 3s - loss: 0.4286 - val_loss: 0.4157
Epoch 138/600
720/720 - 3s - loss: 0.4291 - val_loss: 0.4167
Epoch 139/600
720/720 - 3s - loss: 0.4296 - val_loss: 0.4136
Epoch 140/600
720/720 - 3s - loss: 0.4288 - val_loss: 0.4137
Epoch 141/600
720/720 - 

Epoch 260/600
720/720 - 3s - loss: 0.4246 - val_loss: 0.4115
Epoch 261/600
720/720 - 3s - loss: 0.4239 - val_loss: 0.4115
Epoch 262/600
720/720 - 3s - loss: 0.4244 - val_loss: 0.4115
Epoch 263/600
720/720 - 3s - loss: 0.4244 - val_loss: 0.4115
Epoch 264/600
720/720 - 3s - loss: 0.4244 - val_loss: 0.4114
Epoch 265/600
720/720 - 3s - loss: 0.4245 - val_loss: 0.4115
Epoch 266/600
720/720 - 3s - loss: 0.4246 - val_loss: 0.4115
Epoch 267/600
720/720 - 3s - loss: 0.4241 - val_loss: 0.4115
Epoch 268/600
720/720 - 3s - loss: 0.4242 - val_loss: 0.4114
Epoch 269/600
720/720 - 3s - loss: 0.4242 - val_loss: 0.4115
Epoch 270/600
720/720 - 3s - loss: 0.4243 - val_loss: 0.4115
Epoch 271/600
720/720 - 3s - loss: 0.4244 - val_loss: 0.4114
Epoch 272/600
720/720 - 3s - loss: 0.4241 - val_loss: 0.4115
Epoch 273/600
720/720 - 3s - loss: 0.4246 - val_loss: 0.4115
Epoch 274/600
720/720 - 3s - loss: 0.4241 - val_loss: 0.4114
Epoch 275/600
720/720 - 3s - loss: 0.4243 - val_loss: 0.4115
Epoch 276/600
720/720 - 

INFO:tensorflow:Assets written to: Models/Model1\assets


INFO:tensorflow:Assets written to: Models/Model1\assets


[[8 0 0 ... 0 0 0]
 [8 0 0 ... 0 0 0]
 [8 0 0 ... 0 0 0]
 ...
 [8 0 0 ... 0 0 0]
 [8 0 0 ... 0 0 0]
 [8 0 0 ... 0 0 0]]
finding activity nr 2
[[8]
 [8]
 [8]
 ...
 [8]
 [8]
 [8]]
finding activity nr 3
[[8 6]
 [8 6]
 [8 6]
 ...
 [8 6]
 [8 6]
 [8 6]]
finding activity nr 4
[[ 8  6 10]
 [ 8  6 10]
 [ 8  6 10]
 ...
 [ 8  6 10]
 [ 8  6 10]
 [ 8  6 10]]
finding activity nr 5
[[ 8  6 10  4]
 [ 8  6 10  4]
 [ 8  6 10  4]
 ...
 [ 8  6 10  4]
 [ 8  6 10  4]
 [ 8  6 10  4]]
finding activity nr 6
[[ 8  6 10  4 12]
 [ 8  6 10  4  5]
 [ 8  6 10  4  3]
 ...
 [ 8  6 10  4  7]
 [ 8  6 10  4  7]
 [ 8  6 10  4 12]]
finding activity nr 7
[[ 8  6 10  4 12  3]
 [ 8  6 10  4  5  3]
 [ 8  6 10  4  3  7]
 ...
 [ 8  6 10  4  7  1]
 [ 8  6 10  4  7 12]
 [ 8  6 10  4 12  3]]
finding activity nr 8
[[ 8  6 10 ... 12  3  5]
 [ 8  6 10 ...  5  3  1]
 [ 8  6 10 ...  3  7  5]
 ...
 [ 8  6 10 ...  7  1  3]
 [ 8  6 10 ...  7 12  1]
 [ 8  6 10 ... 12  3  1]]
finding activity nr 9
[[ 8  6 10 ...  3  5  7]
 [ 8  6 10 ...  3  

In [10]:
fit_models_prefix10('Model2')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                7552      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 26)                858       
Total params: 8,410
Trainable params: 8,410
Non-trainable params: 0
_________________________________________________________________
Epoch 1/600
1066/1066 - 6s - loss: 0.8187 - val_loss: 0.4156
Epoch 2/600
1066/1066 - 4s - loss: 0.4729 - val_loss: 0.3663
Epoch 3/600
1066/1066 - 4s - loss: 0.4244 - val_loss: 0.3563
Epoch 4/600
1066/1066 - 4s - loss: 0.3901 - val_loss: 0.3426
Epoch 5/600
1066/1066 - 4s - loss: 0.3935 - val_loss: 0.3400
Epoch 6/600
1066/1066 - 4s - loss: 0.3558 - val_loss: 0.3329
Epoch 7/600
1066

Epoch 121/600
1066/1066 - 4s - loss: 0.2957 - val_loss: 0.2921
Epoch 122/600
1066/1066 - 4s - loss: 0.2969 - val_loss: 0.2929
Epoch 123/600
1066/1066 - 4s - loss: 0.2942 - val_loss: 0.2907
Epoch 124/600
1066/1066 - 4s - loss: 0.2942 - val_loss: 0.2902
Epoch 125/600
1066/1066 - 4s - loss: 0.2939 - val_loss: 0.2899
Epoch 126/600
1066/1066 - 4s - loss: 0.2942 - val_loss: 0.2902
Epoch 127/600
1066/1066 - 4s - loss: 0.2942 - val_loss: 0.2897
Epoch 128/600
1066/1066 - 4s - loss: 0.2942 - val_loss: 0.2905
Epoch 129/600
1066/1066 - 4s - loss: 0.2939 - val_loss: 0.2923
Epoch 130/600
1066/1066 - 4s - loss: 0.2941 - val_loss: 0.2907
Epoch 131/600
1066/1066 - 4s - loss: 0.2940 - val_loss: 0.2903
Epoch 132/600
1066/1066 - 4s - loss: 0.2939 - val_loss: 0.2898
Epoch 133/600
1066/1066 - 4s - loss: 0.2941 - val_loss: 0.2899
Epoch 134/600
1066/1066 - 4s - loss: 0.2941 - val_loss: 0.2922
Epoch 135/600
1066/1066 - 4s - loss: 0.2940 - val_loss: 0.2901
Epoch 136/600
1066/1066 - 4s - loss: 0.2938 - val_loss:

1066/1066 - 4s - loss: 0.2905 - val_loss: 0.2873
Epoch 252/600
1066/1066 - 4s - loss: 0.2906 - val_loss: 0.2873
Epoch 253/600
1066/1066 - 4s - loss: 0.2905 - val_loss: 0.2873
Epoch 254/600
1066/1066 - 4s - loss: 0.2905 - val_loss: 0.2873
Epoch 255/600
1066/1066 - 4s - loss: 0.2907 - val_loss: 0.2873
Epoch 256/600
1066/1066 - 4s - loss: 0.2905 - val_loss: 0.2873
Epoch 257/600
1066/1066 - 4s - loss: 0.2904 - val_loss: 0.2872
Epoch 258/600
1066/1066 - 4s - loss: 0.2904 - val_loss: 0.2872
Epoch 259/600
1066/1066 - 4s - loss: 0.2904 - val_loss: 0.2872
Epoch 260/600
1066/1066 - 4s - loss: 0.2904 - val_loss: 0.2872
Epoch 261/600
1066/1066 - 4s - loss: 0.2907 - val_loss: 0.2873
Epoch 262/600
1066/1066 - 4s - loss: 0.2904 - val_loss: 0.2872
Epoch 263/600
1066/1066 - 4s - loss: 0.2904 - val_loss: 0.2872
Epoch 264/600
1066/1066 - 4s - loss: 0.2905 - val_loss: 0.2872
Epoch 265/600
1066/1066 - 4s - loss: 0.2905 - val_loss: 0.2872
Epoch 266/600
1066/1066 - 4s - loss: 0.2906 - val_loss: 0.2872
Epoch 

INFO:tensorflow:Assets written to: Models/Model2\assets


INFO:tensorflow:Assets written to: Models/Model2\assets


[[15  0  0 ...  0  0  0]
 [15  0  0 ...  0  0  0]
 [15  0  0 ...  0  0  0]
 ...
 [15  0  0 ...  0  0  0]
 [15  0  0 ...  0  0  0]
 [15  0  0 ...  0  0  0]]
finding activity nr 2
[[15]
 [15]
 [15]
 ...
 [15]
 [15]
 [15]]
finding activity nr 3
[[15 19]
 [15 19]
 [15 19]
 ...
 [15 19]
 [15 19]
 [15 19]]
finding activity nr 4
[[15 19 23]
 [15 19 23]
 [15 19 23]
 ...
 [15 19 23]
 [15 19 23]
 [15 19 23]]
finding activity nr 5
[[15 19 23  2]
 [15 19 23  2]
 [15 19 23 21]
 ...
 [15 19 23 21]
 [15 19 23 21]
 [15 19 23  2]]
finding activity nr 6
[[15 19 23  2 13]
 [15 19 23  2 13]
 [15 19 23 21 13]
 ...
 [15 19 23 21 13]
 [15 19 23 21 13]
 [15 19 23  2 13]]
finding activity nr 7
[[15 19 23  2 13 24]
 [15 19 23  2 13 24]
 [15 19 23 21 13  8]
 ...
 [15 19 23 21 13  8]
 [15 19 23 21 13  8]
 [15 19 23  2 13  8]]
finding activity nr 8
[[15 19 23 ... 13 24  5]
 [15 19 23 ... 13 24  5]
 [15 19 23 ... 13  8  5]
 ...
 [15 19 23 ... 13  8  5]
 [15 19 23 ... 13  8  5]
 [15 19 23 ... 13  8  5]]
finding acti

In [11]:
fit_models_full_prefix('Model3')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                7680      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 27)                891       
Total params: 8,571
Trainable params: 8,571
Non-trainable params: 0
_________________________________________________________________
Epoch 1/600
1071/1071 - 6s - loss: 0.9891 - val_loss: 0.5268
Epoch 2/600
1071/1071 - 5s - loss: 0.6221 - val_loss: 0.4427
Epoch 3/600
1071/1071 - 5s - loss: 0.5651 - val_loss: 0.4111
Epoch 4/600
1071/1071 - 5s - loss: 0.5210 - val_loss: 0.3987
Epoch 5/600
1071/1071 - 5s - loss: 0.4580 - val_loss: 0.3803
Epoch 6/600
1071/1071 - 5s - loss: 0.5121 - val_loss: 0.4221
Epoch 7/600
1071

Epoch 121/600
1071/1071 - 5s - loss: 0.3019 - val_loss: 0.2987
Epoch 122/600
1071/1071 - 4s - loss: 0.3024 - val_loss: 0.2979
Epoch 123/600
1071/1071 - 5s - loss: 0.3045 - val_loss: 0.2971
Epoch 124/600
1071/1071 - 5s - loss: 0.3016 - val_loss: 0.2962
Epoch 125/600
1071/1071 - 5s - loss: 0.3023 - val_loss: 0.2969
Epoch 126/600
1071/1071 - 5s - loss: 0.3024 - val_loss: 0.2975
Epoch 127/600
1071/1071 - 5s - loss: 0.3020 - val_loss: 0.2973
Epoch 128/600
1071/1071 - 4s - loss: 0.3031 - val_loss: 0.2964
Epoch 129/600
1071/1071 - 5s - loss: 0.3018 - val_loss: 0.2976
Epoch 130/600
1071/1071 - 4s - loss: 0.3015 - val_loss: 0.2971
Epoch 131/600
1071/1071 - 5s - loss: 0.2994 - val_loss: 0.2951
Epoch 132/600
1071/1071 - 5s - loss: 0.2994 - val_loss: 0.2944
Epoch 133/600
1071/1071 - 5s - loss: 0.2994 - val_loss: 0.2945
Epoch 134/600
1071/1071 - 4s - loss: 0.2994 - val_loss: 0.2947
Epoch 135/600
1071/1071 - 4s - loss: 0.2991 - val_loss: 0.2945
Epoch 136/600
1071/1071 - 5s - loss: 0.2995 - val_loss:

1071/1071 - 5s - loss: 0.2958 - val_loss: 0.2918
Epoch 252/600
1071/1071 - 5s - loss: 0.2958 - val_loss: 0.2918
Epoch 253/600
1071/1071 - 5s - loss: 0.2957 - val_loss: 0.2918
Epoch 254/600
1071/1071 - 5s - loss: 0.2959 - val_loss: 0.2918
Epoch 255/600
1071/1071 - 5s - loss: 0.2958 - val_loss: 0.2918
Epoch 256/600
1071/1071 - 5s - loss: 0.2959 - val_loss: 0.2918
Epoch 257/600
1071/1071 - 5s - loss: 0.2958 - val_loss: 0.2918
Epoch 258/600
1071/1071 - 5s - loss: 0.2958 - val_loss: 0.2918
Epoch 259/600
1071/1071 - 5s - loss: 0.2957 - val_loss: 0.2918
Epoch 260/600
1071/1071 - 5s - loss: 0.2958 - val_loss: 0.2918
Epoch 261/600
1071/1071 - 4s - loss: 0.2958 - val_loss: 0.2918
Epoch 262/600
1071/1071 - 5s - loss: 0.2958 - val_loss: 0.2918
Epoch 263/600
1071/1071 - 5s - loss: 0.2959 - val_loss: 0.2918
Epoch 264/600
1071/1071 - 5s - loss: 0.2960 - val_loss: 0.2918
Epoch 265/600
1071/1071 - 5s - loss: 0.2959 - val_loss: 0.2918
Epoch 266/600
1071/1071 - 5s - loss: 0.2956 - val_loss: 0.2918
Epoch 

INFO:tensorflow:Assets written to: Models/Model3\assets


INFO:tensorflow:Assets written to: Models/Model3\assets


[[16  0  0 ...  0  0  0]
 [16  0  0 ...  0  0  0]
 [16  0  0 ...  0  0  0]
 ...
 [16  0  0 ...  0  0  0]
 [16  0  0 ...  0  0  0]
 [16  0  0 ...  0  0  0]]
finding activity nr 2
[[16]
 [16]
 [16]
 ...
 [16]
 [16]
 [16]]
finding activity nr 3
[[16 20]
 [16 20]
 [16 20]
 ...
 [16 20]
 [16 20]
 [16 20]]
finding activity nr 4
[[16 20 23]
 [16 20 23]
 [16 20 23]
 ...
 [16 20 23]
 [16 20 23]
 [16 20 23]]
finding activity nr 5
[[16 20 23 22]
 [16 20 23 22]
 [16 20 23  2]
 ...
 [16 20 23  2]
 [16 20 23 22]
 [16 20 23 22]]
finding activity nr 6
[[16 20 23 22 14]
 [16 20 23 22 14]
 [16 20 23  2 14]
 ...
 [16 20 23  2 14]
 [16 20 23 22 14]
 [16 20 23 22 14]]
finding activity nr 7
[[16 20 23 22 14  7]
 [16 20 23 22 14 25]
 [16 20 23  2 14 25]
 ...
 [16 20 23  2 14  7]
 [16 20 23 22 14  7]
 [16 20 23 22 14  7]]
finding activity nr 8
[[16 20 23 ... 14  7  5]
 [16 20 23 ... 14 25  5]
 [16 20 23 ... 14 25  5]
 ...
 [16 20 23 ... 14  7  5]
 [16 20 23 ... 14  7  5]
 [16 20 23 ... 14  7  5]]
finding acti

In [5]:
fit_models_prefix10('Model4')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                6528      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 18)                594       
Total params: 7,122
Trainable params: 7,122
Non-trainable params: 0
_________________________________________________________________
Epoch 1/600
891/891 - 16s - loss: 0.8352 - val_loss: 0.4380
Epoch 2/600
891/891 - 4s - loss: 0.4654 - val_loss: 0.3979
Epoch 3/600
891/891 - 4s - loss: 0.4276 - val_loss: 0.3760
Epoch 4/600
891/891 - 4s - loss: 0.4100 - val_loss: 0.3638
Epoch 5/600
891/891 - 4s - loss: 0.3993 - val_loss: 0.3552
Epoch 6/600
891/891 - 4s - loss: 0.3928 - val_loss: 0.3550
Epoch 7/600
891/891 - 4s - lo

Epoch 125/600
891/891 - 4s - loss: 0.2972 - val_loss: 0.2865
Epoch 126/600
891/891 - 4s - loss: 0.2966 - val_loss: 0.2871
Epoch 127/600
891/891 - 4s - loss: 0.2970 - val_loss: 0.2866
Epoch 128/600
891/891 - 4s - loss: 0.2975 - val_loss: 0.2867
Epoch 129/600
891/891 - 4s - loss: 0.2971 - val_loss: 0.2874
Epoch 130/600
891/891 - 4s - loss: 0.2972 - val_loss: 0.2860
Epoch 131/600
891/891 - 4s - loss: 0.2972 - val_loss: 0.2869
Epoch 132/600
891/891 - 4s - loss: 0.2973 - val_loss: 0.2865
Epoch 133/600
891/891 - 4s - loss: 0.2972 - val_loss: 0.2872
Epoch 134/600
891/891 - 4s - loss: 0.2971 - val_loss: 0.2860
Epoch 135/600
891/891 - 4s - loss: 0.2967 - val_loss: 0.2860
Epoch 136/600
891/891 - 4s - loss: 0.2969 - val_loss: 0.2876
Epoch 137/600
891/891 - 4s - loss: 0.2967 - val_loss: 0.2865
Epoch 138/600
891/891 - 4s - loss: 0.2967 - val_loss: 0.2863
Epoch 139/600
891/891 - 4s - loss: 0.2969 - val_loss: 0.2862
Epoch 140/600
891/891 - 4s - loss: 0.2967 - val_loss: 0.2860
Epoch 141/600
891/891 - 

Epoch 260/600
891/891 - 4s - loss: 0.2940 - val_loss: 0.2843


INFO:tensorflow:Assets written to: Models/Model4\assets


INFO:tensorflow:Assets written to: Models/Model4\assets


[[13  0  0 ...  0  0  0]
 [13  0  0 ...  0  0  0]
 [13  0  0 ...  0  0  0]
 ...
 [13  0  0 ...  0  0  0]
 [13  0  0 ...  0  0  0]
 [13  0  0 ...  0  0  0]]
finding activity nr 2
[[13]
 [13]
 [13]
 ...
 [13]
 [13]
 [13]]
finding activity nr 3
[[13  7]
 [13  7]
 [13  7]
 ...
 [13  7]
 [13  7]
 [13  7]]
finding activity nr 4
[[13  7 15]
 [13  7 15]
 [13  7 15]
 ...
 [13  7 15]
 [13  7 15]
 [13  7 15]]
finding activity nr 5
[[13  7 15  4]
 [13  7 15  4]
 [13  7 15  4]
 ...
 [13  7 15  4]
 [13  7 15  4]
 [13  7 15  4]]
finding activity nr 6
[[13  7 15  4  5]
 [13  7 15  4  5]
 [13  7 15  4  5]
 ...
 [13  7 15  4 10]
 [13  7 15  4 10]
 [13  7 15  4 10]]
finding activity nr 7
[[13  7 15  4  5 16]
 [13  7 15  4  5 16]
 [13  7 15  4  5 16]
 ...
 [13  7 15  4 10  5]
 [13  7 15  4 10  5]
 [13  7 15  4 10  5]]
finding activity nr 8
[[13  7 15 ...  5 16 18]
 [13  7 15 ...  5 16 18]
 [13  7 15 ...  5 16 18]
 ...
 [13  7 15 ... 10  5 16]
 [13  7 15 ... 10  5 16]
 [13  7 15 ... 10  5 16]]
finding acti

In [6]:
fit_models_prefix10('Model5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                7296      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 24)                792       
Total params: 8,088
Trainable params: 8,088
Non-trainable params: 0
_________________________________________________________________
Epoch 1/600
1382/1382 - 8s - loss: 0.9421 - val_loss: 0.4899
Epoch 2/600
1382/1382 - 6s - loss: 0.5329 - val_loss: 0.3952
Epoch 3/600
1382/1382 - 6s - loss: 0.4620 - val_loss: 0.3695
Epoch 4/600
1382/1382 - 6s - loss: 0.4276 - val_loss: 0.3511
Epoch 5/600
1382/1382 - 6s - loss: 0.4089 - val_loss: 0.3439
Epoch 6/600
1382/1382 - 6s - loss: 0.3978 - val_loss: 0.3365
Epoch 7/600
1382

Epoch 121/600
1382/1382 - 7s - loss: 0.2590 - val_loss: 0.2442
Epoch 122/600
1382/1382 - 7s - loss: 0.2594 - val_loss: 0.2390
Epoch 123/600
1382/1382 - 7s - loss: 0.2605 - val_loss: 0.2411
Epoch 124/600
1382/1382 - 7s - loss: 0.2595 - val_loss: 0.2377
Epoch 125/600
1382/1382 - 8s - loss: 0.2595 - val_loss: 0.2398
Epoch 126/600
1382/1382 - 8s - loss: 0.2589 - val_loss: 0.2360
Epoch 127/600
1382/1382 - 7s - loss: 0.2589 - val_loss: 0.2359
Epoch 128/600
1382/1382 - 7s - loss: 0.2595 - val_loss: 0.2446
Epoch 129/600
1382/1382 - 7s - loss: 0.2592 - val_loss: 0.2419
Epoch 130/600
1382/1382 - 8s - loss: 0.2584 - val_loss: 0.2398
Epoch 131/600
1382/1382 - 7s - loss: 0.2592 - val_loss: 0.2482
Epoch 132/600
1382/1382 - 7s - loss: 0.2577 - val_loss: 0.2407
Epoch 133/600
1382/1382 - 8s - loss: 0.2590 - val_loss: 0.2429
Epoch 134/600
1382/1382 - 8s - loss: 0.2581 - val_loss: 0.2445
Epoch 135/600
1382/1382 - 7s - loss: 0.2583 - val_loss: 0.2465
Epoch 136/600
1382/1382 - 7s - loss: 0.2586 - val_loss:

1382/1382 - 6s - loss: 0.2376 - val_loss: 0.2260
Epoch 252/600
1382/1382 - 6s - loss: 0.2376 - val_loss: 0.2259
Epoch 253/600
1382/1382 - 6s - loss: 0.2382 - val_loss: 0.2261
Epoch 254/600
1382/1382 - 6s - loss: 0.2379 - val_loss: 0.2255
Epoch 255/600
1382/1382 - 7s - loss: 0.2378 - val_loss: 0.2262
Epoch 256/600
1382/1382 - 6s - loss: 0.2383 - val_loss: 0.2256
Epoch 257/600
1382/1382 - 7s - loss: 0.2381 - val_loss: 0.2261
Epoch 258/600
1382/1382 - 6s - loss: 0.2378 - val_loss: 0.2260
Epoch 259/600
1382/1382 - 6s - loss: 0.2380 - val_loss: 0.2256
Epoch 260/600
1382/1382 - 6s - loss: 0.2379 - val_loss: 0.2259
Epoch 261/600
1382/1382 - 6s - loss: 0.2378 - val_loss: 0.2258
Epoch 262/600
1382/1382 - 6s - loss: 0.2377 - val_loss: 0.2258
Epoch 263/600
1382/1382 - 6s - loss: 0.2377 - val_loss: 0.2258
Epoch 264/600
1382/1382 - 6s - loss: 0.2377 - val_loss: 0.2257
Epoch 265/600
1382/1382 - 7s - loss: 0.2379 - val_loss: 0.2256
Epoch 266/600
1382/1382 - 8s - loss: 0.2370 - val_loss: 0.2255
Epoch 

INFO:tensorflow:Assets written to: Models/Model5\assets


INFO:tensorflow:Assets written to: Models/Model5\assets


[[16  0  0 ...  0  0  0]
 [16  0  0 ...  0  0  0]
 [16  0  0 ...  0  0  0]
 ...
 [16  0  0 ...  0  0  0]
 [16  0  0 ...  0  0  0]
 [16  0  0 ...  0  0  0]]
finding activity nr 2
[[16]
 [16]
 [16]
 ...
 [16]
 [16]
 [16]]
finding activity nr 3
[[16  6]
 [16  6]
 [16  6]
 ...
 [16  6]
 [16  6]
 [16  6]]
finding activity nr 4
[[16  6 21]
 [16  6 21]
 [16  6 21]
 ...
 [16  6 21]
 [16  6 21]
 [16  6 21]]
finding activity nr 5
[[16  6 21 24]
 [16  6 21 24]
 [16  6 21 24]
 ...
 [16  6 21 24]
 [16  6 21 24]
 [16  6 21 24]]
finding activity nr 6
[[16  6 21 24 20]
 [16  6 21 24 20]
 [16  6 21 24 20]
 ...
 [16  6 21 24 20]
 [16  6 21 24 20]
 [16  6 21 24 20]]
finding activity nr 7
[[16  6 21 24 20 23]
 [16  6 21 24 20 23]
 [16  6 21 24 20 23]
 ...
 [16  6 21 24 20 23]
 [16  6 21 24 20 23]
 [16  6 21 24 20 23]]
finding activity nr 8
[[16  6 21 ... 20 23 19]
 [16  6 21 ... 20 23 19]
 [16  6 21 ... 20 23 17]
 ...
 [16  6 21 ... 20 23 17]
 [16  6 21 ... 20 23 17]
 [16  6 21 ... 20 23 19]]
finding acti

In [29]:
fit_models_prefix10_withloops('Model6')

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 32)                6272      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
Total params: 6,800
Trainable params: 6,800
Non-trainable params: 0
_________________________________________________________________
Epoch 1/600
1427/1427 - 8s - loss: 0.5996 - val_loss: 0.3048
Epoch 2/600
1427/1427 - 7s - loss: 0.3154 - val_loss: 0.2805
Epoch 3/600
1427/1427 - 6s - loss: 0.2920 - val_loss: 0.2714
Epoch 4/600
1427/1427 - 7s - loss: 0.2797 - val_loss: 0.2566
Epoch 5/600
1427/1427 - 7s - loss: 0.2717 - val_loss: 0.2578
Epoch 6/600
1427/1427 - 7s - loss: 0.2650 - val_loss: 0.2454
Epoch 7/600
1427

Epoch 121/600
1427/1427 - 8s - loss: 0.2158 - val_loss: 0.2116
Epoch 122/600
1427/1427 - 9s - loss: 0.2155 - val_loss: 0.2110
Epoch 123/600
1427/1427 - 8s - loss: 0.2157 - val_loss: 0.2109
Epoch 124/600
1427/1427 - 8s - loss: 0.2157 - val_loss: 0.2114
Epoch 125/600
1427/1427 - 9s - loss: 0.2156 - val_loss: 0.2109
Epoch 126/600
1427/1427 - 8s - loss: 0.2155 - val_loss: 0.2105
Epoch 127/600
1427/1427 - 8s - loss: 0.2156 - val_loss: 0.2109
Epoch 128/600
1427/1427 - 8s - loss: 0.2155 - val_loss: 0.2110
Epoch 129/600
1427/1427 - 8s - loss: 0.2155 - val_loss: 0.2116
Epoch 130/600
1427/1427 - 9s - loss: 0.2156 - val_loss: 0.2110
Epoch 131/600
1427/1427 - 8s - loss: 0.2157 - val_loss: 0.2110
Epoch 132/600
1427/1427 - 9s - loss: 0.2156 - val_loss: 0.2108
Epoch 133/600
1427/1427 - 9s - loss: 0.2156 - val_loss: 0.2105
Epoch 134/600
1427/1427 - 8s - loss: 0.2155 - val_loss: 0.2106
Epoch 135/600
1427/1427 - 10s - loss: 0.2153 - val_loss: 0.2106
Epoch 136/600
1427/1427 - 9s - loss: 0.2152 - val_loss

1427/1427 - 8s - loss: 0.2132 - val_loss: 0.2093
Epoch 252/600
1427/1427 - 8s - loss: 0.2132 - val_loss: 0.2093
Epoch 253/600
1427/1427 - 8s - loss: 0.2132 - val_loss: 0.2092
Epoch 254/600
1427/1427 - 8s - loss: 0.2132 - val_loss: 0.2093
Epoch 255/600
1427/1427 - 9s - loss: 0.2132 - val_loss: 0.2093
Epoch 256/600
1427/1427 - 9s - loss: 0.2132 - val_loss: 0.2093
Epoch 257/600
1427/1427 - 8s - loss: 0.2133 - val_loss: 0.2093
Epoch 258/600
1427/1427 - 9s - loss: 0.2134 - val_loss: 0.2093
Epoch 259/600
1427/1427 - 9s - loss: 0.2132 - val_loss: 0.2092
Epoch 260/600
1427/1427 - 8s - loss: 0.2133 - val_loss: 0.2093
Epoch 261/600
1427/1427 - 8s - loss: 0.2132 - val_loss: 0.2093
Epoch 262/600
1427/1427 - 8s - loss: 0.2132 - val_loss: 0.2093
Epoch 263/600
1427/1427 - 8s - loss: 0.2131 - val_loss: 0.2093
Epoch 264/600
1427/1427 - 8s - loss: 0.2133 - val_loss: 0.2093
Epoch 265/600
1427/1427 - 9s - loss: 0.2132 - val_loss: 0.2093
Epoch 266/600
1427/1427 - 7s - loss: 0.2134 - val_loss: 0.2093
Epoch 

INFO:tensorflow:Assets written to: Models/Model6\assets


INFO:tensorflow:Assets written to: Models/Model6\assets


[[9 0 0 ... 0 0 0]
 [9 0 0 ... 0 0 0]
 [9 0 0 ... 0 0 0]
 ...
 [9 0 0 ... 0 0 0]
 [9 0 0 ... 0 0 0]
 [9 0 0 ... 0 0 0]]
finding activity nr 2
[[9]
 [9]
 [9]
 ...
 [9]
 [9]
 [9]]
finding activity nr 3
[[9 5]
 [9 5]
 [9 5]
 ...
 [9 5]
 [9 5]
 [9 5]]
finding activity nr 4
[[ 9  5 12]
 [ 9  5 12]
 [ 9  5 12]
 ...
 [ 9  5 12]
 [ 9  5 12]
 [ 9  5 12]]
finding activity nr 5
[[ 9  5 12  2]
 [ 9  5 12  2]
 [ 9  5 12  2]
 ...
 [ 9  5 12  2]
 [ 9  5 12  2]
 [ 9  5 12  2]]
finding activity nr 6
[[ 9  5 12  2  7]
 [ 9  5 12  2  7]
 [ 9  5 12  2  7]
 ...
 [ 9  5 12  2  7]
 [ 9  5 12  2  7]
 [ 9  5 12  2  7]]
finding activity nr 7
[[ 9  5 12  2  7  4]
 [ 9  5 12  2  7  2]
 [ 9  5 12  2  7  2]
 ...
 [ 9  5 12  2  7  4]
 [ 9  5 12  2  7  2]
 [ 9  5 12  2  7  4]]
finding activity nr 8
[[ 9  5 12 ...  7  4 13]
 [ 9  5 12 ...  7  2  7]
 [ 9  5 12 ...  7  2  7]
 ...
 [ 9  5 12 ...  7  4 13]
 [ 9  5 12 ...  7  2  7]
 [ 9  5 12 ...  7  4 13]]
finding activity nr 9
[[ 9  5 12 ...  4 13  8]
 [ 9  5 12 ...  2  

finding activity nr 50
[[ 9  5 12 ... 16  6 16]
 [ 9  5 12 ... 16  6 16]
 [ 9  5 12 ...  7  4 13]
 ...
 [ 9  5 12 ...  7  2  7]
 [ 9  5 12 ... 11  8 11]
 [ 9  5 12 ...  3  3  3]]
finding activity nr 51
[[ 9  5 12 ...  6 16  6]
 [ 9  5 12 ...  6 16  6]
 [ 9  5 12 ...  4 13  8]
 ...
 [ 9  5 12 ...  2  7  2]
 [ 9  5 12 ...  8 11 10]
 [ 9  5 12 ...  3  3 15]]
finding activity nr 52
[[ 9  5 12 ... 16  6 14]
 [ 9  5 12 ... 16  6 14]
 [ 9  5 12 ... 13  8 11]
 ...
 [ 9  5 12 ...  7  2  7]
 [ 9  5 12 ... 11 10  1]
 [ 9  5 12 ...  3 15  3]]
finding activity nr 53
[[ 9  5 12 ...  6 14 15]
 [ 9  5 12 ...  6 14 15]
 [ 9  5 12 ...  8 11  8]
 ...
 [ 9  5 12 ...  2  7  2]
 [ 9  5 12 ... 10  1 16]
 [ 9  5 12 ... 15  3 12]]
finding activity nr 54
[[ 9  5 12 ... 14 15  3]
 [ 9  5 12 ... 14 15  3]
 [ 9  5 12 ... 11  8 11]
 ...
 [ 9  5 12 ...  7  2  7]
 [ 9  5 12 ...  1 16  6]
 [ 9  5 12 ...  3 12  2]]
finding activity nr 55
[[ 9  5 12 ... 15  3  3]
 [ 9  5 12 ... 15  3  3]
 [ 9  5 12 ...  8 11  8]
 ...
 [

In [30]:
fit_models_prefix10_complex('MoreComplex')

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 32)                5888      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 13)                429       
Total params: 6,317
Trainable params: 6,317
Non-trainable params: 0
_________________________________________________________________
Epoch 1/600
4713/4713 - 26s - loss: 0.7688 - val_loss: 0.6407
Epoch 2/600
4713/4713 - 25s - loss: 0.6699 - val_loss: 0.6276
Epoch 3/600
4713/4713 - 26s - loss: 0.6605 - val_loss: 0.6357
Epoch 4/600
4713/4713 - 26s - loss: 0.6570 - val_loss: 0.6220
Epoch 5/600
4713/4713 - 26s - loss: 0.6543 - val_loss: 0.6439
Epoch 6/600
4713/4713 - 27s - loss: 0.6529 - val_loss: 0.6165
Epoch 7/60

Epoch 119/600
4713/4713 - 19s - loss: 0.5709 - val_loss: 0.5542
Epoch 120/600
4713/4713 - 19s - loss: 0.5709 - val_loss: 0.5543
Epoch 121/600
4713/4713 - 21s - loss: 0.5710 - val_loss: 0.5535
Epoch 122/600
4713/4713 - 19s - loss: 0.5709 - val_loss: 0.5538
Epoch 123/600
4713/4713 - 19s - loss: 0.5707 - val_loss: 0.5545
Epoch 124/600
4713/4713 - 20s - loss: 0.5705 - val_loss: 0.5535
Epoch 125/600
4713/4713 - 20s - loss: 0.5706 - val_loss: 0.5540
Epoch 126/600
4713/4713 - 21s - loss: 0.5706 - val_loss: 0.5535
Epoch 127/600
4713/4713 - 25s - loss: 0.5707 - val_loss: 0.5542
Epoch 128/600
4713/4713 - 23s - loss: 0.5706 - val_loss: 0.5533
Epoch 129/600
4713/4713 - 21s - loss: 0.5703 - val_loss: 0.5547
Epoch 130/600
4713/4713 - 19s - loss: 0.5705 - val_loss: 0.5539
Epoch 131/600
4713/4713 - 21s - loss: 0.5699 - val_loss: 0.5542
Epoch 132/600
4713/4713 - 19s - loss: 0.5702 - val_loss: 0.5548
Epoch 133/600
4713/4713 - 20s - loss: 0.5701 - val_loss: 0.5533
Epoch 134/600
4713/4713 - 19s - loss: 0.

4713/4713 - 21s - loss: 0.5661 - val_loss: 0.5510
Epoch 248/600
4713/4713 - 20s - loss: 0.5660 - val_loss: 0.5510
Epoch 249/600
4713/4713 - 20s - loss: 0.5662 - val_loss: 0.5510
Epoch 250/600
4713/4713 - 20s - loss: 0.5661 - val_loss: 0.5510
Epoch 251/600
4713/4713 - 18s - loss: 0.5662 - val_loss: 0.5510
Epoch 252/600
4713/4713 - 18s - loss: 0.5661 - val_loss: 0.5510
Epoch 253/600
4713/4713 - 18s - loss: 0.5660 - val_loss: 0.5510
Epoch 254/600
4713/4713 - 17s - loss: 0.5662 - val_loss: 0.5510
Epoch 255/600
4713/4713 - 18s - loss: 0.5663 - val_loss: 0.5510
Epoch 256/600
4713/4713 - 17s - loss: 0.5662 - val_loss: 0.5511
Epoch 257/600
4713/4713 - 18s - loss: 0.5660 - val_loss: 0.5510
Epoch 258/600
4713/4713 - 17s - loss: 0.5661 - val_loss: 0.5510
Epoch 259/600
4713/4713 - 19s - loss: 0.5662 - val_loss: 0.5510
Epoch 260/600
4713/4713 - 20s - loss: 0.5661 - val_loss: 0.5510
Epoch 261/600
4713/4713 - 19s - loss: 0.5659 - val_loss: 0.5510
Epoch 262/600
4713/4713 - 27s - loss: 0.5664 - val_los

INFO:tensorflow:Assets written to: Models/MoreComplex\assets


INFO:tensorflow:Assets written to: Models/MoreComplex\assets


[[12  0  0 ...  0  0  0]
 [12  0  0 ...  0  0  0]
 [12  0  0 ...  0  0  0]
 ...
 [12  0  0 ...  0  0  0]
 [12  0  0 ...  0  0  0]
 [12  0  0 ...  0  0  0]]
finding activity nr 2
[[12]
 [12]
 [12]
 ...
 [12]
 [12]
 [12]]
finding activity nr 3
[[12  8]
 [12  8]
 [12  8]
 ...
 [12  8]
 [12  8]
 [12  8]]
finding activity nr 4
[[12  8  7]
 [12  8  7]
 [12  8  7]
 ...
 [12  8  7]
 [12  8  7]
 [12  8  7]]
finding activity nr 5
[[12  8  7  1]
 [12  8  7  1]
 [12  8  7  1]
 ...
 [12  8  7  1]
 [12  8  7  1]
 [12  8  7  1]]
finding activity nr 6
[[12  8  7  1 13]
 [12  8  7  1  3]
 [12  8  7  1  4]
 ...
 [12  8  7  1  3]
 [12  8  7  1 13]
 [12  8  7  1  4]]
finding activity nr 7
[[12  8  7  1 13  2]
 [12  8  7  1  3  4]
 [12  8  7  1  4 10]
 ...
 [12  8  7  1  3  4]
 [12  8  7  1 13  3]
 [12  8  7  1  4 10]]
finding activity nr 8
[[12  8  7 ... 13  2  6]
 [12  8  7 ...  3  4  3]
 [12  8  7 ...  4 10  3]
 ...
 [12  8  7 ...  3  4 10]
 [12  8  7 ... 13  3  9]
 [12  8  7 ...  4 10  3]]
finding acti